In [7]:
'''
Last edited: 10/26/20

Improving the script to run on npx using csky tutorials


Stacking sensitivity for different time windows and different gamma (spectral indices). Gamma will range from 2-3 and the time windows will range from 10^(-2) seconds to 10^5 seconds. Stacking sensitivity will be performed with Csky likelihood software.  
'''

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from astropy.time import Time
import argparse
import histlite as hl
import csky as cy

#Building/loading ps tracks from analysis directory, switch selections to MESEDataSpecs.mesc_7yr for cascades
repo = cy.selections.Repository()
ana_dir = cy.utils.ensure_dir('/home/mkovacevich/FRB_analysis/cascades_ana')

repo = cy.selections.Repository()
ana = cy.analysis.Analysis(repo, cy.selections.MESEDataSpecs.mesc_7yr ,dir=ana_dir)
'''
parser = argparse.ArgumentParser(description='Process sensitivities for FRB catalog over livetime of MESC 7yr dataset')
parser.add_argument('--gamma',type=float,help='Spectral indice for E')
parser.add_argument('--dt', type=float, help='time-window in units of seconds')
parser.add_argument('--seed',type=float,help ='keep track of the total number of jobs submitted')
args = parser.parse_args()
'''

#Calculating sensitivities, 3sigma at 90% and discovery potential as functions of different time windows for 1 FRB
#The time windows will range from ~1 ms to 10^4 seconds (following previous times windows)
cy.CONF['ana'] = ana
cy.CONF['mp_cpus'] = 4

gamma = 3
time_window = 1.
n_trials = 10000
n_sigs = np.r_[2:10:1, 10:30.1:2]

trials_dir = cy.utils.ensure_dir('/home/mkovacevich/FRB_analysis/trials')
sig_dir = cy.utils.ensure_dir('{}/sig'.format(trials_dir))
bg_dir = cy.utils.ensure_dir('{}/bg'.format(trials_dir))

#Source times and equitorial coordinates of FRB single bursts
source_times = ['2018-03-11T04:11:54.800','2018-03-09T02:49:32.990000','2018-03-01T07:34:19.760000','2017-12-09T20:34:23.500000','2017-09-22T11:22:23.400000','2017-08-27T16:20:18','2017-01-07T20:05:45.139000','2016-06-08T03:53:01.088000','2016-04-10T08:33:39.680000','2016-03-17T09:00:36.530000','2016-01-02T08:28:39.374000','2015-12-30T16:15:46.525000','2015-12-06T06:17:52.778000','2015-08-07T17:53:55.830000','2015-06-10T05:26:59.396000','2015-04-18T04:29:06.657000','2015-02-15T20:41:41.714000','2014-05-14T17:14:11.060000','2013-11-04T18:04:11.200000','2013-07-29T09:01:51.190000','2013-06-28T03:58:00.178000','2013-06-26T14:55:59.771000','2012-10-02T13:09:18.436000','2012-01-27T08:11:21.725000','2011-07-03T18:59:40.607000','2011-06-26T21:33:17.477000','2011-05-23T15:06:19.700000','2011-02-20T01:55:48.096000']
FRB_ra_deg = [322.89,321.18,93.18,237.60,322.46,12.33,170.69,114.17,130.35,118.45,339.70,145.21,290.35,340.63,161.11,109.15,274.36,338.52,101.04,205.34,135.76,246.77,273.70,348.77,352.71,315.93,326.30,338.66]
FRB_dec_deg = [-56.26,-32.02,4.56,-45.83,-6.01,-64.45,-4.98,-39.20,6.08,-28.39,-29.82,-2.55,-3.87,-54.92,-39.91,-18.99,-3.10,-11.69,-50.72,-4.0,3.44,-6.54,-84.80,-17.57,-1.13,-43.26,.016,-11.60]

#Converting date and times to mjd time
src_t = Time(source_times)
mjd_source_time = src_t.mjd

#loading analysis object to get mjd of time data-set and events
a = ana.anas[0]

#good_indices represents an array that tracks the indices of FRB events that fall within livetime of MESC 7 yr
FRB_mjd_time, FRB_ra_rad, FRB_dec_rad, FRB_time_window, good_indices = [],[],[],[],[]


#Now, we are going to simplify the above arrays to only include FRBs that burst during the livetime of MESC 7 yr
for i, time in enumerate(mjd_source_time):
    if float(time) >= min(a.data['mjd']) and float(time) <= max(a.data['mjd']):
        FRB_mjd_time.append(time)
        good_indices.append(i)
        
#converting degrees to radians and an array of time windows that can be used in the configuration below - this is
#necessary for stacking
FRB_ra_rad = [np.radians(FRB_ra_deg[i]) for i in good_indices]
FRB_dec_rad = [np.radians(FRB_dec_deg[i]) for i in good_indices]
FRB_time_window = np.ones_like(FRB_ra_rad)*time_window/86400.


#For time windows less than a second, the median bg TS will be 0 so when calculating the sensitivity we allow
#bg.median() = 0 to save computational time
def do_background_trials(N=n_trials):
    # get trial runner
    src = cy.sources(FRB_ra_rad, FRB_dec_rad, mjd = FRB_mjd_time, sigma_t = np.zeros_like(FRB_ra_rad), t_100 = FRB_time_window)
    conf = {'extended':True, 'space':"ps",'time':"transient",'sig':"transient",'flux': cy.hyp.PowerLawFlux(3)}
    tr = cy.get_trial_runner(conf, src = src, ana=ana)
    # run trials
    trials = tr.get_many_fits(N,logging=False)
    # save to disk
    dir = cy.utils.ensure_dir('{}/gamma/{:+04.0f}'.format(bg_dir,gamma))
    filename = '{}/bg_trials_dt_{:04f}.npy'.format(dir, time_window)
    print('->', filename)
    # notice: trials.as_array is a numpy structured array, not a cy.utils.Arrays
    np.save(filename, trials.as_array)
        
n_sigs = np.r_[2:10:1, 10:30.1:2]
def do_signal_trials(N=):
    # get trial runner
    src = cy.sources(FRB_ra_rad, FRB_dec_rad, mjd = FRB_mjd_time, sigma_t = np.zeros_like(FRB_ra_rad), t_100 = FRB_time_window)
    conf = {'extended':True, 'space':"ps",'time':"transient",'sig':"transient",'flux': cy.hyp.PowerLawFlux(3)}
    tr = cy.get_trial_runner(conf, src = src, ana=ana)
    # run trials
    trials = tr.get_many_fits(N, n_sig,logging=False)
    # save to disk
    dir = cy.utils.ensure_dir('{}/gamma/{:+04.0f}'.format(sig_dir,gamma))
    filename = '{}/sig_trials_dt_{:06f}.npy'.format(dir, time_window)
    print('->', filename)
    # notice: trials.as_array is a numpy structured array, not a cy.utils.Arrays
    np.save(filename, trials.as_array)

do_background_trials(N=n_trials)
do_signal_trials(n_sig,N=n_trials)

Setting up Analysis for:
MESC_2010_2016
Setting up MESC_2010_2016...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2013_MC.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC79_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2011_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2012_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2013_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2014_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2015_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/IC86_2016_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/GRL/IC79_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/GRL/IC86_2011_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/GRL/IC86_2012_exp.npy ...
Reading /data/ana/analyses/mese_cascades/version-001-p02/GRL/

KeyboardInterrupt: 

In [ ]:
'''
Posting previous script that was run for bg and signal trials below (to keep track)
'''

'''
Stacking sensitivity for different time windows and different gamma (spectral indices). Gamma will range from 2-3 and the time windows will range from 10^(-2) seconds to 10^5 seconds. Stacking sensitivity will be performed with Csky likelihood software.  
'''

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from astropy.time import Time
import argparse
import histlite as hl
import csky as cy

#Building/loading ps tracks from analysis directory, switch selections to MESEDataSpecs.mesc_7yr for cascades
repo = cy.selections.Repository()
ana_dir = cy.utils.ensure_dir('/home/mkovacevich/FRB_analysis/cascades_ana')

repo = cy.selections.Repository()
ana = cy.analysis.Analysis(repo, cy.selections.MESEDataSpecs.mesc_7yr ,dir=ana_dir)

parser = argparse.ArgumentParser(description='Process sensitivities for FRB catalog over livetime of MESC 7yr dataset')
parser.add_argument('--gamma',type=float,help='Spectral indice for E')
parser.add_argument('--dt', type=float, help='time-window in units of seconds')
parser.add_argument('--sigma',type=float, help = 'which discovery potential (3,4 or 5 sigma @ 90%) to look at')
args = parser.parse_args()

#Calculating sensitivities, 3sigma at 90% and discovery potential as functions of different time windows for 1 FRB
#The time windows will range from ~1 ms to 10^4 seconds (following previous times windows)
cy.CONF['ana'] = ana
cy.CONF['mp_cpus'] = 4

#Source times and equitorial coordinates of FRB single bursts
source_times = ['2018-03-11T04:11:54.800','2018-03-09T02:49:32.990000','2018-03-01T07:34:19.760000','2017-12-09T20:34:23.500000','2017-09-22T11:22:23.400000','2017-08-27T16:20:18','2017-01-07T20:05:45.139000','2016-06-08T03:53:01.088000','2016-04-10T08:33:39.680000','2016-03-17T09:00:36.530000','2016-01-02T08:28:39.374000','2015-12-30T16:15:46.525000','2015-12-06T06:17:52.778000','2015-08-07T17:53:55.830000','2015-06-10T05:26:59.396000','2015-04-18T04:29:06.657000','2015-02-15T20:41:41.714000','2014-05-14T17:14:11.060000','2013-11-04T18:04:11.200000','2013-07-29T09:01:51.190000','2013-06-28T03:58:00.178000','2013-06-26T14:55:59.771000','2012-10-02T13:09:18.436000','2012-01-27T08:11:21.725000','2011-07-03T18:59:40.607000','2011-06-26T21:33:17.477000','2011-05-23T15:06:19.700000','2011-02-20T01:55:48.096000']
FRB_ra_deg = [322.89,321.18,93.18,237.60,322.46,12.33,170.69,114.17,130.35,118.45,339.70,145.21,290.35,340.63,161.11,109.15,274.36,338.52,101.04,205.34,135.76,246.77,273.70,348.77,352.71,315.93,326.30,338.66]
FRB_dec_deg = [-56.26,-32.02,4.56,-45.83,-6.01,-64.45,-4.98,-39.20,6.08,-28.39,-29.82,-2.55,-3.87,-54.92,-39.91,-18.99,-3.10,-11.69,-50.72,-4.0,3.44,-6.54,-84.80,-17.57,-1.13,-43.26,.016,-11.60]

#Converting date and times to mjd time
src_t = Time(source_times)
mjd_source_time = src_t.mjd

#loading analysis object to get mjd of time data-set and events
a = ana.anas[0]

#good_indices represents an array that tracks the indices of FRB events that fall within livetime of MESC 7 yr
FRB_mjd_time, FRB_ra_rad, FRB_dec_rad, FRB_time_window, good_indices = [],[],[],[],[]


#Now, we are going to simplify the above arrays to only include FRBs that burst during the livetime of MESC 7 yr
for i, time in enumerate(mjd_source_time):
    if float(time) >= min(a.data['mjd']) and float(time) <= max(a.data['mjd']):
        FRB_mjd_time.append(time)
        good_indices.append(i)
        
FRB_ra_rad = [np.radians(FRB_ra_deg[i]) for i in good_indices]
FRB_dec_rad = [np.radians(FRB_dec_deg[i]) for i in good_indices]
FRB_time_window = np.ones_like(FRB_ra_rad)*args.dt/86400.

        

    #dt represents the time window in seconds over seconds in a day
    src = cy.sources(FRB_ra_rad, FRB_dec_rad, mjd = FRB_mjd_time, sigma_t = np.zeros_like(FRB_ra_rad), t_100 = FRB_time_window)
    conf = {'extended':True, 'space':"ps",'time':"transient",'sig':"transient",'flux': cy.hyp.PowerLawFlux(args.gamma)}
    tr = cy.get_trial_runner(conf, src = src, ana=ana)

    n_bg_trials = 1e7
    n_trials =1e4

    #Creating background TS distribution, using TSD rather than Chi2 due to small time window and low event rate for MESC
    #TSD uses trials/data rather than Chi2 fit
    bg = cy.dists.TSD(tr.get_many_fits(n_bg_trials))
    np.save('bg_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma), bg, allow_pickle = True)
                    
    if args.sigma == 0:
        #signal trials to get sensitivity
        sig = tr.find_n_sig(bg.median(), .9, n_sig_step=5, batch_size=n_trials,tol=.05) 
        np.save('sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),sig,allow_pickle=True)

        sens = tr.to_E2dNdE(sig,E0=1e5)
        np.save('sens_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),sens,allow_pickle=True)
                    
    elif args.sigma == 3:
        #three sigma discovery potential
        three_sig = tr.find_n_sig(bg.isf_nsigma(args.sigma), 0.9, n_sig_step=5, batch_size=10000, tol=.05, fit=False)
        np.save('3_sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),three_sig,allow_pickle=True)

        three_sig_disc = tr.to_E2dNdE(three_sig,E0=1e5)
        np.save('3_sig_disc_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),three_sig_disc,allow_pickle=True)
    elif args.sigma == 4:
        #four sigma discovery potential
        four_sig = tr.find_n_sig(bg.isf_nsigma(args.sigma), 0.9, n_sig_step=5, batch_size=10000, tol=.05, fit=False)
        np.save('4_sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),four_sig,allow_pickle=True)

        four_sig_disc = tr.to_E2dNdE(four_sig,E0=1e5)
        np.save('4_sig_disc_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),four_sig_disc,allow_pickle=True)
    elif args.sigma == 5:
        #five sigma discovery potential
        five_sig = tr.find_n_sig(bg.isf_nsigma(args.sigma), 0.9, n_sig_step=5, batch_size=10000, tol=.05, fit=False)
        np.save('5_sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),five_sig,allow_pickle=True)

        five_sig_disc = tr.to_E2dNdE(five_sig,E0=1e5)
        np.save('5_sig_disc_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),five_sig_disc,allow_pickle=True)

elif args.dt <= 1.:
    #dt represents the time window in seconds over seconds in a day
    src = cy.sources(FRB_ra_rad, FRB_dec_rad, mjd = FRB_mjd_time, sigma_t = np.zeros_like(FRB_ra_rad), t_100 = FRB_time_window)
    conf = {'extended':True, 'space':"ps",'time':"transient",'sig':"transient",'flux': cy.hyp.PowerLawFlux(args.gamma)}
    tr = cy.get_trial_runner(conf, src = src, ana=ana)

    n_bg_trials = 1e7
    n_trials =1e4

    #Creating background TS distribution, using TSD rather than Chi2 due to small time window and low event rate for MESC
    #TSD uses trials/data rather than Chi2 fit
    bg = cy.dists.TSD(tr.get_many_fits(n_bg_trials))
    np.save('bg_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma), bg, allow_pickle = True)

    #signal trials to get sensitivity
    if args.sigma == 0:
        #signal trials to get sensitivity
        sig = tr.find_n_sig(0., .9, n_sig_step=5, batch_size=n_trials,tol=.05) 
        np.save('sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),sig,allow_pickle=True)

        sens = tr.to_E2dNdE(sig,E0=1e5)
        np.save('sens_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),sens,allow_pickle=True)
    if args.sigma == 3:
        #three sigma discovery potential
        three_sig = tr.find_n_sig(bg.isf_nsigma(args.sigma), 0.9, n_sig_step=5, batch_size=n_trials, tol=.05, fit=False)
        np.save('3_sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),three_sig,allow_pickle=True)

        three_sig_disc = tr.to_E2dNdE(three_sig,E0=1e5)
        np.save('3_sig_disc_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),three_sig_disc,allow_pickle=True)
    elif args.sigma == 4:
        #four sigma discovery potential
        four_sig = tr.find_n_sig(bg.isf_nsigma(args.sigma), 0.9, n_sig_step=5, batch_size=n_trials, tol=.05, fit=False)
        np.save('4_sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),four_sig,allow_pickle=True)

        four_sig_disc = tr.to_E2dNdE(four_sig,E0=1e5)
        np.save('4_sig_disc_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),four_sig_disc,allow_pickle=True)
    elif args.sigma == 5:
        #five sigma discovery potential
        five_sig = tr.find_n_sig(bg.isf_nsigma(args.sigma), 0.9, n_sig_step=5, batch_size=n_trials, tol=.05, fit=False)
        np.save('5_sig_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),five_sig,allow_pickle=True)

        five_sig_disc = tr.to_E2dNdE(five_sig,E0=1e5)
        np.save('5_sig_disc_gamma_'+str(args.gamma)+'dt'+str(args.dt)+'sigma'+str(args.sigma),five_sig_disc,allow_pickle=True)
